<a href="https://colab.research.google.com/github/Sercan-dev/bitcoin_price_trend_prediction/blob/main/bitcoin_price_trend_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
import yfinance as yf
import numpy as np
import pandas as pd
import math
import datetime
import os
import time
import matplotlib.pyplot as plt
import xgboost as xgb

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import classification_report
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


# Sentiment data

Data is from the previous Twitter sentiment analysis I've done.
[Github](https://github.com/Sercan-dev/Bitcoin_sentiment_analysis/blob/8d5b9b83204dc4bb13af59ee40776a40874ec60f/bitcoin_sentiment_analysis.ipynb)

In [2]:
#Get the sentiment analysis data
from google.colab import files

df = pd.read_csv('sentiment_results.csv')
df

,created_at,new_tweet,pred,label,score
0,2022-07-08 18:00:10+00:00,computerart creativecoding generativeart bitco...,1,POSITIVE,0.997290
1,2022-07-08 18:00:10+00:00,bitcoin leads best week since october crypto d...,1,POSITIVE,0.994598
2,2022-07-08 18:00:11+00:00,crypto traders lose 280 million following bitc...,0,NEGATIVE,0.997662
3,2022-07-08 18:00:12+00:00,gxs usdt volume increase detected spike 171 90...,1,POSITIVE,0.991317
4,2022-07-08 18:00:13+00:00,voyager bankruptcy last straw crypto hodlers c...,0,NEGATIVE,0.999421
...,...,...,...,...,...
57158,2022-07-10 11:48:41+00:00,wealthy persons blueprint 1 invest first spend...,1,POSITIVE,0.998444
57159,2022-07-10 11:48:43+00:00,100trillionusd holding 20k support would great...,1,POSITIVE,0.996198
57160,2022-07-10 11:48:45+00:00,actually bitcoin ledger bitcoin asset lives le...,0,NEGATIVE,0.989842
57161,2022-07-10 11:48:46+00:00,rackmom found bitcoin user vault location join...,1,POSITIVE,0.998292


# Next let's get the Price quote of Bitcoin

In [3]:
#Get the price quote of bitcoin
Price = yf.download('BTC-USD',  start="2022-07-08", end="2022-07-11", interval = "1h")

Price = pd.DataFrame(Price,columns=['Close'])

Price = Price.truncate(before='2022-07-08 18:00:10 UTC', after='2022-07-10 11:48:48 UTC')



[*********************100%***********************]  1 of 1 completed


In [4]:
#Get the returns(%) of bitcoin price by the hour

Price['Returns'] = Price.Close.pct_change()

In [5]:
Price.head(10)

,Close,Returns
2022-07-08 19:00:00+00:00,21810.101562,NaN
2022-07-08 20:00:00+00:00,21858.921875,0.002238
2022-07-08 21:00:00+00:00,21809.523438,-0.002260
2022-07-08 22:00:00+00:00,21776.236328,-0.001526
2022-07-08 23:00:00+00:00,21753.833984,-0.001029
2022-07-09 00:00:00+00:00,21534.480469,-0.010083
2022-07-09 01:00:00+00:00,21501.798828,-0.001518
2022-07-09 02:00:00+00:00,21559.103516,0.002665
2022-07-09 03:00:00+00:00,21593.500000,0.001595
2022-07-09 04:00:00+00:00,21549.691406,-0.002029


In [6]:
Price.isnull().sum()

Close      0
Returns    1
dtype: int64

In [7]:
#set index to datetime format

df['date'] = pd.to_datetime(df['created_at'])
df = df.set_index('date') 

In [8]:
df

,created_at,new_tweet,pred,label,score
date,,,,,
2022-07-08 18:00:10+00:00,2022-07-08 18:00:10+00:00,computerart creativecoding generativeart bitco...,1,POSITIVE,0.997290
2022-07-08 18:00:10+00:00,2022-07-08 18:00:10+00:00,bitcoin leads best week since october crypto d...,1,POSITIVE,0.994598
2022-07-08 18:00:11+00:00,2022-07-08 18:00:11+00:00,crypto traders lose 280 million following bitc...,0,NEGATIVE,0.997662
2022-07-08 18:00:12+00:00,2022-07-08 18:00:12+00:00,gxs usdt volume increase detected spike 171 90...,1,POSITIVE,0.991317
2022-07-08 18:00:13+00:00,2022-07-08 18:00:13+00:00,voyager bankruptcy last straw crypto hodlers c...,0,NEGATIVE,0.999421
...,...,...,...,...,...
2022-07-10 11:48:41+00:00,2022-07-10 11:48:41+00:00,wealthy persons blueprint 1 invest first spend...,1,POSITIVE,0.998444
2022-07-10 11:48:43+00:00,2022-07-10 11:48:43+00:00,100trillionusd holding 20k support would great...,1,POSITIVE,0.996198
2022-07-10 11:48:45+00:00,2022-07-10 11:48:45+00:00,actually bitcoin ledger bitcoin asset lives le...,0,NEGATIVE,0.989842


In [9]:
## Resample Timeframe to hourly and get the mean value of the tweets for a given hour

sentiment = pd.DataFrame(df.resample('H')['pred'].mean())

In [10]:
sentiment.head(10)

,pred
date,
2022-07-08 18:00:00+00:00,0.584579
2022-07-08 19:00:00+00:00,0.656280
2022-07-08 20:00:00+00:00,0.624411
2022-07-08 21:00:00+00:00,0.573610
2022-07-08 22:00:00+00:00,0.595164
2022-07-08 23:00:00+00:00,0.588672
2022-07-09 00:00:00+00:00,0.629910
2022-07-09 01:00:00+00:00,0.631500
2022-07-09 02:00:00+00:00,0.606481


In [11]:
# Merge the two dataframes

price_df = Price.join(sentiment)

In [12]:
price_df.head(10)

,Close,Returns,pred
2022-07-08 19:00:00+00:00,21810.101562,NaN,0.656280
2022-07-08 20:00:00+00:00,21858.921875,0.002238,0.624411
2022-07-08 21:00:00+00:00,21809.523438,-0.002260,0.573610
2022-07-08 22:00:00+00:00,21776.236328,-0.001526,0.595164
2022-07-08 23:00:00+00:00,21753.833984,-0.001029,0.588672
2022-07-09 00:00:00+00:00,21534.480469,-0.010083,0.629910
2022-07-09 01:00:00+00:00,21501.798828,-0.001518,0.631500
2022-07-09 02:00:00+00:00,21559.103516,0.002665,0.606481
2022-07-09 03:00:00+00:00,21593.500000,0.001595,0.635652
2022-07-09 04:00:00+00:00,21549.691406,-0.002029,0.631579


# Next let's create the target: The column for price trend 1 = price went up, 0 = price went down

In [13]:
#Create price trend up or down column

UP = 1

DOWN = 0

price_df['Price Trend'] = np.where(
    price_df['Returns'] > 0, UP, DOWN
)

In [14]:
price_df.head()

,Close,Returns,pred,Price Trend
2022-07-08 19:00:00+00:00,21810.101562,NaN,0.656280,0
2022-07-08 20:00:00+00:00,21858.921875,0.002238,0.624411,1
2022-07-08 21:00:00+00:00,21809.523438,-0.002260,0.573610,0
2022-07-08 22:00:00+00:00,21776.236328,-0.001526,0.595164,0
2022-07-08 23:00:00+00:00,21753.833984,-0.001029,0.588672,0


In [15]:
price_df = price_df.drop(['Close'], axis=1)

In [16]:
price_df.dropna(inplace=True)

In [17]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40 entries, 2022-07-08 20:00:00+00:00 to 2022-07-10 11:00:00+00:00
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Returns      40 non-null     float64
 1   pred         40 non-null     float64
 2   Price Trend  40 non-null     int64  
dtypes: float64(2), int64(1)
memory usage: 1.2 KB


In [18]:
attributes = ['pred', 'Price Trend']
df_model= price_df[attributes].copy()

In [19]:
## define the the dependent variable y and independent variable X
attributes = ['pred']
X = df_model[attributes]
y = df_model['Price Trend']


##Divide data into train set an test set
train_pct_index = int(0.7 * len(X))
X_train, X_test = X[:train_pct_index], X[train_pct_index:]
y_train, y_test = y[:train_pct_index], y[train_pct_index:]


##Fit the model
model = XGBClassifier()
model.fit(X_train, y_train)
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)


##Print the accuracy of the model
acc_train = accuracy_score(y_train, pred_train)
acc_test = accuracy_score(y_test, pred_test)
print('****Train Results****')
print("Accuracy: {:.4%}".format(acc_train))
print('****Test Results****')
print("Accuracy: {:.4%}".format(acc_test))

****Train Results****
Accuracy: 92.8571%
****Test Results****
Accuracy: 50.0000%


In [20]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

matrix_train = confusion_matrix(y_train, pred_train)
matrix_test = confusion_matrix(y_test, pred_test)

print(matrix_train)
print(matrix_test)

report_train = classification_report(y_train, pred_train)
report_test = classification_report(y_test, pred_test)

print(report_train)
print(report_test)


print(model.get_booster().feature_names)

[[13  1]
 [ 1 13]]
[[3 3]
 [3 3]]
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        14
           1       0.93      0.93      0.93        14

    accuracy                           0.93        28
   macro avg       0.93      0.93      0.93        28
weighted avg       0.93      0.93      0.93        28

              precision    recall  f1-score   support

           0       0.50      0.50      0.50         6
           1       0.50      0.50      0.50         6

    accuracy                           0.50        12
   macro avg       0.50      0.50      0.50        12
weighted avg       0.50      0.50      0.50        12

['pred']


# Conclusion

There is obviously not enough data left when resampling the data into hourly timesteps and calculating the mean value of the tweets sentiment in a given hour to train the model properly.

That's why I didn't go into the fine tuning of the model.

I will definitely try this model again with a much larger data set to find out if it's possible to predict price movements of Bitcoin with the help of Twitter sentiments.